In [9]:
%load_ext lab_black

In [117]:
import re
import urllib.request

from bs4 import BeautifulSoup, NavigableString

In [118]:
user_agent = "Mozilla/5.0 (Windows; U; Windows NT 5.1; en-US; rv:1.9.0.7) Gecko/2009021910 Firefox/3.0.7"
url = "https://www.alberta.ca/find-shelters.aspx"
headers = {
    "User-Agent": user_agent,
}

request = urllib.request.Request(url, None, headers)
response = urllib.request.urlopen(request)
data = response.read()

shelter_soup = BeautifulSoup(data)

In [119]:
calgary_shelters_start = shelter_soup.find(id="goa-grid11591").find(
    lambda tag: tag.name == "h3" and tag.string == "Calgary"
)

calgary_shelters_end = shelter_soup.find(id="goa-grid11591").find(
    lambda tag: tag.name == "h3" and tag.string == "Edmonton"
)

In [120]:
shelters = []

for sib in calgary_shelters_start.next_siblings:
    if sib == calgary_shelters_end:
        break
    elif sib == "\n":
        continue
    shelters.append(sib)

In [121]:
shelter_names = [shelter.find("a", target="_blank").string for shelter in shelters]
shelter_addresses = [
    shelter.find(string=re.compile("^\n[\d]+|^\n[\w]+\s[\w]+,\s[\d]+"))
    for shelter in shelters
]
shelter_addresses = [str(s).strip().replace("\xa0", " ") for s in shelter_addresses]

In [122]:
shelters_a = {
    name: address for (name, address) in zip(shelter_names, shelter_addresses)
}
shelters_a

{'Alpha House Shelter': '203 15 Avenue SE',
 'Brenda’s House': '1921 28 Street SW',
 'Calgary Drop-In & Rehab Centre': '1 Dermot Baldwin Way SE',
 'Inn From the Cold': '110 11 Avenue SE',
 'YW Calgary Emergency Shelter': '1715 17 Avenue SE',
 'Mustard Seed Foothills Shelter': '7025 44 Street SE',
 'Salvation Army Centre of Hope': '420 9 Avenue SE',
 'Salvation Army Women’s Integrated Supportive Housing (WISH)': 'Wagner Place, 3013 15 Avenue SW'}

In [126]:
import pandas as pd

df = pd.read_csv(
    "https://open.alberta.ca/dataset/47f82be8-af8d-4994-8a97-2252d7643ff5/resource/b7080b66-25ea-4c30-ac47-02b64353637f/download/2013-2022-emergency-shelter-occupancy-machine-readable.csv"
)

/var/folders/sy/lsfn4dnx0x75hhnpd76qp_c40000gn/T/ipykernel_40359/2367520071.py:2: DtypeWarning: Columns (6,7) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(


In [166]:
df.loc[(df["City"] == "Calgary") & (df["YEAR"] == 2021) & (df["Capacity"] != "0")]

,Date,City,ShelterType,ShelterName,Organization,Shelter,Capacity,Overnight,Daytime,YEAR,MONTH
120596,1/1/2021,Calgary,Adult Emergency,Hillhurst Centre,Calgary Drop-In Centre,Hillhurst Centre,33,33,,2021,1
120597,1/1/2021,Calgary,Adult Emergency,Wagner Centre,Calgary Salvation Army,Wagner Centre (Women),12,0,,2021,1
120598,1/1/2021,Calgary,Adult Emergency,Centre of Hope,Calgary Salvation Army,Centre of Hope (Men),80,14,,2021,1
120599,1/1/2021,Calgary,Intox,Alpha House,Alpha House,Intox,45,43,,2021,1
120600,1/1/2021,Calgary,Adult Emergency,Mustard Seed,Mustard Seed Calgary,Non-Intox,153,115,,2021,1
...,...,...,...,...,...,...,...,...,...,...,...
158663,12/31/2021,Calgary,Family Emergency,Inn From the Cold - Scattered Sites,Inn from the Cold,Scattered Sites,36,19,,2021,12
158669,12/31/2021,Calgary,COVID19 Social Distancing Measures,Riverfront - Combined,Calgary Drop-In Centre,COVID19 Social Distancing Measures,659,584,,2021,12
158671,12/31/2021,Calgary,COVID19 Expanded Shelter,Mustard Seed - First Alliance,Mustard Seed Calgary,COVID19 Expanded Shelter,120,60,,2021,12
158692,12/31/2021,Calgary,COVID19 Isolation Site,River Front - Isolation,Calgary Drop-In Centre,,7,0,,2021,12
